# Cellpy tweaking
Notebook for testing ideas and check stuff.

*Remember to run the notebook in the correct environment (for conda, issue* `conda install nb_conda` *to be able to switch environments).*

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import cellpy.parameters.prms as prms
from cellpy import cellreader

In [ ]:
from cellpy import log
log.setup_logging(default_level="DEBUG")

## Some digging into the cellpy structure

In [ ]:
# print settings 
prm_dicts = [d for d in dir(prms) if not d.startswith("_")]

for d in prm_dicts:
    if isinstance(getattr(prms,d), (dict, )) and not d.startswith("excel"):
        print(f"\n--------------------{d}----------------------")
        for variable in getattr(prms, d):
            print(f"{variable}: ", end="")
            print(eval("prms.%s['%s']" % (d, variable)))
    

## Defining filenames etc

In [ ]:
current_file_path = os.getcwd()
relative_test_data_dir = "../testdata"
test_data_dir = os.path.abspath(os.path.join(current_file_path, relative_test_data_dir))
test_data_dir_raw = os.path.join(test_data_dir, "data")
test_res_file = "20160805_test001_45_cc_01.res"
test_res_file_full = os.path.join(test_data_dir_raw,test_res_file)
test_data_dir_out = os.path.join(test_data_dir, "out")
test_data_dir_cellpy = os.path.join(test_data_dir, "hdf5")
test_cellpy_file = "20160805_test001_45_cc.h5"
test_cellpy_file_tmp = "tmpfile.h5"
test_cellpy_file_full = os.path.join(test_data_dir_cellpy,test_cellpy_file)
test_cellpy_file_tmp_full = os.path.join(test_data_dir_cellpy,test_cellpy_file_tmp)
test_run_name = "20160805_test001_45_cc"

In [ ]:
assert os.path.isfile(test_res_file_full)
assert os.path.isfile(test_cellpy_file_full)
assert os.path.isdir(test_data_dir)
# os.listdir(test_data_dir)

## Loading and looking at what we got

In [ ]:
celldata = cellreader.CellpyData()
#celldata.from_raw(test_res_file_full)
#celldata.set_mass(0.34)
celldata.load(test_cellpy_file_full)

In [ ]:
data = celldata.dataset

In [ ]:
for x in dir(data):
    if not x.startswith("__"):
        if not x in ["dfsummary", "dfdata", "step_table"]:
            print(f"{x}: {getattr(data,x)}")

In [ ]:
data.dfsummary.head()

In [ ]:
data.dfdata.head()

In [ ]:
data.step_table.head()

In [ ]:
data.dfsummary_made = True

In [ ]:
celldata.make_summary()

In [ ]:
print(data.dfsummary_made)

dfsummary_made is wrong in the cellpyfile

## Task 1: capacity voltage curves
Would like to make it possible to chose how the voltage curves are presented.

In [ ]:
# lets see what cycles we have
cycles = celldata.get_cycle_numbers()
print(cycles)

In [ ]:
capacity, voltage = celldata.get_cap(cycle=(1,2))
capacity_f, voltage_f = celldata.get_cap(cycle=(1,2), method="forth", shift=250)
capacity_ff, voltage_ff = celldata.get_cap(cycle=(1,2), method="forth-and-forth")
capacity_shift, voltage_shift = celldata.get_cap(cycle=(1,2), shift=1000.0)

plt.plot(capacity, voltage, label="bf")
plt.plot(capacity_f, voltage_f, label="f")
plt.plot(capacity_ff, voltage_ff, label="ff")
plt.plot(capacity_shift, voltage_shift, label="s")
plt.legend()